<a href="https://colab.research.google.com/github/joaquinhojman/ArbolesDeDecision/blob/main/ModeloLightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


from sklearn.model_selection import train_test_split

In [ ]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'
url_test_kaggle = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test_kaggle.csv'

data_train = pd.read_csv(url_train)
data_test = pd.read_csv(url_test)
data_test_kaggle = pd.read_csv(url_test_kaggle)

In [ ]:
# Separamos los set de datos (data_train y data_test) de sus columnas 'Target'
# 'x' son los datos, 'y' es el target:
#   --> x_train son los datos con los que entrenar
#   --> y_train son las soluciones con las que comparar lo entrenado con x_train
#   --> x_test son los datos con los que probar
#   --> y_test son las soluciones con las que comparar lo probado con x_test
x_train = data_train.iloc[:, :-1]
y_train = data_train.iloc[:, -1]
x_test = data_test.iloc[:, :-1]
y_test = data_test.iloc[:, -1]

#Esto lo hizo chechu y esta bien, no cambiar.


In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'binary_logloss',              
    'num_leaves': 62,
    'learning_rate': 0.05,
    'feature_fraction': 0.22,
    'boosting': 'gbdt','min_samples_split':100
 } 

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)

preds = gbm.predict(x_test, num_iteration=gbm.best_iteration)

[1]	valid_0's binary_logloss: 0.665674
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.649214
[3]	valid_0's binary_logloss: 0.632452
[4]	valid_0's binary_logloss: 0.6299
[5]	valid_0's binary_logloss: 0.618848
[6]	valid_0's binary_logloss: 0.617165
[7]	valid_0's binary_logloss: 0.616189
[8]	valid_0's binary_logloss: 0.603878
[9]	valid_0's binary_logloss: 0.589202
[10]	valid_0's binary_logloss: 0.583186
[11]	valid_0's binary_logloss: 0.583538
[12]	valid_0's binary_logloss: 0.573916
[13]	valid_0's binary_logloss: 0.564314
[14]	valid_0's binary_logloss: 0.561508
[15]	valid_0's binary_logloss: 0.551687
[16]	valid_0's binary_logloss: 0.546888
[17]	valid_0's binary_logloss: 0.541927
[18]	valid_0's binary_logloss: 0.539029
[19]	valid_0's binary_logloss: 0.537532
[20]	valid_0's binary_logloss: 0.53402
[21]	valid_0's binary_logloss: 0.528736
[22]	valid_0's binary_logloss: 0.527365
[23]	valid_0's binary_logloss: 0.521714
[24]	valid_0's binary_logloss

In [ ]:

error = log_loss(y_test, preds)
error

0.4283481286477098

In [ ]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [ ]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = preds

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [ ]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.43503971316847356

In [ ]:
# Sacamos el preds_test_agrupado_por_opp
preds_test_kaggle = gbm.predict(data_test_kaggle)

oportunidades_test = data_test_kaggle['Opportunity_ID']
oportunidades_test = oportunidades_test.to_frame()

oportunidades_test['Target'] = preds_test_kaggle

preds_test_kaggle_agrupado_por_opp = oportunidades_test.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_kaggle_agrupado_por_opp = preds_test_kaggle_agrupado_por_opp.reset_index()

preds_test_kaggle_agrupado_por_opp

,Opportunity_ID,Target
0,10689.0,0.614775
1,10690.0,0.527273
2,10691.0,0.383132
3,10692.0,0.519192
4,10693.0,0.649120
...,...,...
1562,12364.0,0.571441
1563,12365.0,0.320641
1564,12366.0,0.244239
1565,12367.0,0.406347


In [ ]:
from google.colab import files

preds_test_kaggle_agrupado_por_opp.to_csv('preds_testlgbm.csv', index=False, header=True)
files.download('preds_testlgbm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
!pip install hyperopt
from hyperopt.pyll.base import scope 
import warnings
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [46]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
space={'max_depth': hp.quniform("max_depth", 3, 12, 1),
    #    'learning_rate' : hp.quniform('learning_rate', 0.001, 0.5, 0.01),
        'min_samples_split' :hp.quniform('min_samples_split',2,200,25),
        'num_leaves': hp.quniform('num_leaves', 62,63,1),
    #    'feature_fraction': hp.quniform('feature_fraction', 0.01,0.1,1),
        'sub_feature': hp.quniform('sub_feature',0.1,1,1),
        'min_data': hp.quniform('min_data', 10,150,1)
    }

def hyperparameter_tuning(space):
 #   params = {
  #      'max_depth':int(space['max_depth']),'min_samples_split' :int(space['min_samples_split']),'num_leaves' :int(space['num_leaves']),
   #     'min_data' :int(space['min_data']),'learning_rate':space['learning_rate'],'feature_fraction':space['feature_fraction'],
    #    'sub_feature':space['sub_feature']
   # } 
    model = lgb.LGBMRegressor(max_depth=int(space['max_depth']),min_samples_split=int(space['min_samples_split']),num_leaves=int(space['num_leaves']),
        min_data=int(space['min_data']),learning_rate=0.05,feature_fraction=0.22,
        sub_feature=space['sub_feature'])
  #  model=lgb.train(params,lgb_train)        
    model.fit(x_train, y_train)                    
    preds = model.predict(x_test)

    logloss = log_loss(y_test, preds)
    print(logloss)
    #change the metric if you like
    return {'loss':logloss, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)


print (best)

0.4683998839782885
0.4351711662648044
0.4480122164621642
0.4293907757111123
0.45149907695358427
0.4317844726414302
0.43373716880296587
0.42389448389704565
0.43813798999109865
0.4312791234637509
0.4268537191004695
0.44933785393030046
0.4430549787646587
0.4271204602008854
0.4739707290059741
0.4373419117915008
0.4483336526505651
0.44155226824096133
0.4136246040507578
0.4460732786920147
0.44029452340317754
0.44737090329150775
0.42527162277138475
0.44113029731832426
0.4404752564697964
0.43325105191504465
0.4216553790543268
0.4260973818782435
0.4288031303774944
0.4656268054738015
0.44246971591665896
0.42319275911412957
0.4401657042081713
0.4360422799910597
0.43851128415625057
0.4103858395835322
0.4236190501628908
0.4193077425839196
0.41604243722467443
0.46176981778976184
0.41917869444063877
0.42572122485830777
0.46160106489575925
0.43796055313290727
0.4418134981035276
0.4340867009196626
0.40591087217686356
0.4101778733343712
0.4280787122595655
0.43256671453121515
0.4356082617888676
0.4339082

In [47]:
#{'max_depth': 10.0, 'min_data': 35.0, 'min_samples_split': 0.0, 'num_leaves': 62.0, 'sub_feature': 0.0}
#best loss: 0.40591087217686356
best

{'max_depth': 10.0,
 'min_data': 35.0,
 'min_samples_split': 0.0,
 'num_leaves': 62.0,
 'sub_feature': 0.0}

In [49]:
lgb_hyper = lgb.LGBMRegressor(max_depth=10,min_samples_split=0,num_leaves=62,
        min_data=35,learning_rate=0.05,feature_fraction=0.22,
        sub_feature=0)
lgb_hyper.fit(x_train, y_train)                    
preds = lgb_hyper.predict(x_test)

logloss = log_loss(y_test, preds)
logloss


0.40591087217686356

In [50]:
#Error de train
preds_train_hyper = lgb_hyper.predict(x_train)
error_train_hyper = log_loss(y_train, preds_train_hyper)
error_train_hyper

0.18104773190577023

In [52]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [53]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = preds

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [54]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.41657399679418755

In [55]:
# Sacamos el preds_test_agrupado_por_opp
preds_test_kaggle_hyper = lgb_hyper.predict(data_test_kaggle)

oportunidades_test = data_test_kaggle['Opportunity_ID']
oportunidades_test = oportunidades_test.to_frame()

oportunidades_test['Target'] = preds_test_kaggle_hyper

preds_test_kaggle_agrupado_por_opp = oportunidades_test.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_kaggle_agrupado_por_opp = preds_test_kaggle_agrupado_por_opp.reset_index()
preds_test_kaggle_agrupado_por_opp['Opportunity_ID'] = preds_test_kaggle_agrupado_por_opp['Opportunity_ID'].astype(int)

preds_test_kaggle_agrupado_por_opp

,Opportunity_ID,Target
0,10689,0.640885
1,10690,0.575505
2,10691,0.386691
3,10692,0.424445
4,10693,0.726168
...,...,...
1562,12364,0.608671
1563,12365,0.351135
1564,12366,0.218267
1565,12367,0.442604


In [56]:
from google.colab import files

preds_test_kaggle_agrupado_por_opp.to_csv('preds_testLIGHTGBM_hyper.csv', index=False, header=True)
files.download('preds_testLIGHTGBM_hyper.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>